# <span style="color:gold">Algoritmi di Crittografia (2023/24)</span> 
## Notebook 6

In [ ]:
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('/home/mauro/.jupyter/custom/custom.css').read()))

Latex definitions
$\DeclareMathOperator*{\prob}{prob}$
$\DeclareMathOperator*{\mod}{mod}$
$\DeclareMathOperator*{\ln}{ln}$
$\def\zn{\mathbf{Z}_n}$
$\def\zp{\mathbf{Z}_p}$
$\def\mcd{\mathrm{MCD}}$

* In questo notebook presentiamo i sistemi asimmetrici proposti da <span style="color:gold">M.O. Rabin</span> e <span style="color:gold">T. El Gamal</span>
* Si tratta di due sistemi che non hanno guadagnato molta importanza, perché "oscurati" da RSA.
* El Gamal è interessante perché mostra come solo un "piccolo passo avanti", rispetto al protocollo di scambio di chiavi proposto da Diffie ed Hellman, avrebbe prodotto il <span style="color:gold">primo vero protocollo asimmetrico</span> (qualche anno prima di RSA)
* Anche la sicurezza del protocollo di El Gamal è dunque basata sul <span style="color:gold">logaritmo discreto</span>
* Il protocollo di Rabin è interessante perché, a differenza di RSA, è "dimostrabilmente" <span style="color:gold">equivalente alla fattorizzazione</span>, su cui basa la propria sicurezza
* Vedremo però che ha un "difetto" che ne ha sbarrato la strada all'impiego pratico

### <span style="color:gold">Il sistema crittografico asimmetrico di Rabin</span>
* Di poco posteriore alla pubblicazione di RSA, il sistema inventato da Michael O. Rabin ha la ragguardevole proprietà teorica di essere <span style="color:gold">equivalente alla fattorizzazione</span>
* Più precisamente, un <span style="color:gold">algoritmo polynomial time</span> in grado di violare il sistema di Rabin potrebbe essere utilizzato per la fattorizzazione di interi (e viceversa)
* Possiede però anche una limitazione, che ne ha impedito la diffusione
* La decifrazione produce infatti <span style="color:gold">quattro possibili candidati</span> come testo in chiaro corrispondente ad un dato ciphertext e, se il testo originale è un numero o una sequenza di bit, questo può essere un serio problema

#### <span style="color:cyan">Un esempio numerico preliminare ralativo al calcolo delle radici quadrate</span>

In [ ]:
from random import randint
from ACLIB.utils import modular_inverse, Euclid, getprime

* Vogliamo calcolare le radici quadrate di un numero $y$ in $Z_n$, dove $n$ è il <span style="color:gold">prodotto di due numeri primi</span>
* Naturalmente il numero deve essere un <span style="color:gold">residuo quadratico modulo $n$</span> (l'equivalente di un quadrato perfetto)
* Per ottenerlo, procediamo a rovescio: scegliamo a caso un numero $x$ e lo eleviamo al quadrato

In [ ]:
p = 13
q = 23
n = p*q
print(n)

In [ ]:
x = randint(1,n-1)
y = x*x%n
print(y)

* Naturalmente, così facendo, una radice di $y$ già la conosciamo, ed è ovviamente $x$. In realtà ne conosciamo subito un'altra, l'opposto di $x$ in $Z_n$ e cioè $n-x$

In [ ]:
(n-x)*(n-x)%n

In [ ]:
print(x,n-x)

* Poiché però $n$ è il prodotto di due primi, le radici quadrate sono in tutto 4
* Come si calcolano le altre 2 (e, in generale, <span style="color:gold">come si calcolano tutte e 4 le radici di un residuo quadratico $y$ arbitrario</span>)?

* Si noti innanzitutto che $y\mod p$ e $y\mod q$ sono a loro volta <span style="color:gold">residui quadratici</span> (modulo $p$ e modulo $q$, rispettivamente)
* Infatti, una radice di $y\mod p$ è $x\mod p$:
$$
((x\mod p)\cdot(x\mod p))\mod p = (x\cdot x)\mod p = ((x\cdot x)\mod n)\mod p = y\mod p
$$
* Analogamente una radice di $y\mod p$ è $x\mod q$.
* Ora, il calcolo delle radici quadrate di $y$ modulo $n$ è facile proprio se conosciamo una radice di $y$ modulo $p$ e una modulo $q$
* Per scopi didattici (e per numeri piccoli) possiamo procedere col metodo <span style="color:gold">brute force</span>

In [ ]:
# Calcolo delle radici di y modulo p
yp = y%p
z1  = 1
while (z1*z1)%p != yp:  
    z1 = z1+1
z2 = p-z1
print(z1,z2)

In [ ]:
# Calcolo delle radici di y modulo q
yq = y%q
w1  = 1
while (w1*w1)%q != yq:   # Se y è un r.q. mod n lo è anche mod q
    w1 = w1+1
w2 = q-w1
print(w1,w2)

* Per calcolare le 4 radici di $y$ mod $n$ calcoliamo preliminarmente le <span style="color:gold">"solite"</span> due quantità
$$
c_1 = q\cdot(q^{-1}\mod p)\\
c_2 = p\cdot(p^{-1}\mod q)
$$

In [ ]:
c1 = q*(modular_inverse(q,p))
c2 = p*(modular_inverse(p,q))

In [ ]:
print(c1%p,c1%q,c2%p,c2%q)

In [ ]:
R1 = (c1*z1+c2*w1)%n      
R2 = (c1*z1-c2*w1)%n               
R3 = (-c1*z1+c2*w1)%n
R4 = (-c1*z1-c2*w1)%n
print(R1,R2,R3,R4)

* Naturalmente, due delle quattro radici sono fra loro <span style="color:gold">congrue modulo $p$</span> e due sono <span style="color:gold">congrue modulo $q$</span>

In [ ]:
from ACLIB.utils import Euclid

In [ ]:
Euclid(R2-R3,n)==q

N.B. Prendere $z_1$ e $w_1$ è <span style="color:gold">arbitrario</span>

In [ ]:
(c1*z2+c2*w1)%n

In [ ]:
print(R1*R1%n)
print(R2*R2%n)
print(R3*R3%n)
print(R4*R4%n)

* Naturalmente due delle 4 radici coincidono con le due che (in questo caso) già conoscevamo

In [ ]:
print(x,n-x)

#### <span style="color:cyan">L'algoritmo generale</span>

* Come per RSA, anche nel protocollo di Rabin la generazione delle chiavi richiede <span style="color:gold">un solo parametro di input</span>, che è la lunghezza in bit, indicata con $N$, dell'intero con cui si effettueranno le riduzioni modulari. 
* Il messaggio $M$ da cifrare dovrà essere <span style="color:gold">un numero minore del modulo</span> ovvero trasformato in un tale numero per mezzo di una <span style="color:gold">funzione invertibile</span>.

<br />

#### <span style="color:cyan">Generazione delle chiavi (Alice)</span>
1. Genera <span style="color:gold">due numeri primi a caso</span>, $p\equiv 3\ (\mod 4)$ e $q\equiv 3\ (\mod 4)$, di lunghezza $N/2$ bit;
2. calcola il <span style="color:gold">prodotto $n=p\cdot q$</span>;
3. diffondi <span style="color:gold">$n$ come chiave pubblica</span> e conserva la coppia <span style="color:gold">$p, q$ come chiave segreta</span>.

* Nota: il fatto che i numeri $p$ e $q$ siano del tipo $4k+3$ non è decisivo. Serve però a rendere più facile il <span style="color:gold">calcolo delle radici quadrate</span> in fase di decifrazione

<br />

#### <span style="color:cyan">Cifratura di un messaggio $M$ (Bob)</span>
1. Si procura la chiave pubblica $n$ di Alice;
2. calcola <span style="color:gold">$C=M^2\ \mathrm{mod}\ n$</span>;
3. invia $C$ ad Alice come messaggio cifrato</li>

<br />

#### <span style="color:cyan">Decifrazione del messaggio $C$ (Alice)</span>

* Ricordiamo preliminarmente che, se $p$ divide $n$, allora vale <span style="color:gold">$(x\mod n)\mod p=x\mod p$</span>. La dimostrazione è semplice e discende direttamente dalla definizione di modulo. 

1. Calcola <span style="color:gold">$M_p=C^{\frac{p+1}{4}}\mod p$</span> e <span style="color:gold">$M_q=C^{\frac{q+1}{4}}\mod q$</span>. Si noti quanto segue.
    * $M_p$ è una delle <span style="color:gold">radici quadrate di $C$</span> modulo $p$ perché 
\begin{eqnarray*}
M_p^2\mod p&=&\left(C^{\frac{p+1}{4}}\mod p\right)^2\mod p\\
&=&C^{\frac{p+1}{2}}\mod p\\
&=&\left(C^{\frac{p-1}{2}}\cdot C\right)\mod p\\
&=&\left((M^2\mod n)^{\frac{p-1}{2}}\cdot C\right)\mod p\\
&=&\left(\left((M^2\mod n)\mod p)\right)^{\frac{p-1}{2}}\cdot C\right)\mod p\\
&=&\left((M^2\mod p)^{\frac{p-1}{2}}\cdot C\right)\mod p\\
&=&\left(M^{p-1}\cdot C\right)\mod p\\
&=&\left(\left(M^{p-1}\mod p\right)\cdot C\right)\mod p\\
&=&\left(1\cdot C\right)\mod p\\
&=&C\mod p
\end{eqnarray*}
    
    * Analogamente $M_q$ è una delle <span style="color:gold">radici quadrate di $C$</span> modulo $q$
   
2. Calcola le seguenti quattro quantità, ciascuna delle quali è una <span style="color:gold">radice quadrata di $C$ modulo $n$</span>
    * $M_1 = \left(\left(q\cdot(q^{-1}\ \mathrm{mod}\ p)\right)M_p + \left(p\cdot(p^{-1}\ \mathrm{mod}\ q)\right)M_q\right)\ \mathrm{mod}\ n=(c_p M_p+c_q M_q)\mod n$
    * $M_2 = n-M_1 = (n-M_1)\mod n = (-M_1 \mod n) = (-c_p M_p-c_q M_q)\mod n$
    * $M_3 = \left(\left(q\cdot(q^{-1}\ \mathrm{mod}\ p)\right)M_p - \left(p\cdot(p^{-1}\ \mathrm{mod}\ q)\right)M_q\right)\ \mathrm{mod}\ n=(c_p M_p-c_q M_q)\mod n$
    * $M_4 = n-M_3 = (-c_p M_p+c_q M_q)\mod n$
3. Uno dei quattro valori è il messaggio originale

* Dimostriamo che si tratta effettivamente delle <span style="color:gold">quattro radici di $C$ modulo $n$</span>
* Per quanto riguarda $M_1$ abbiamo
\begin{eqnarray*}
M_1^2\mod p&=&\left(\left(\left(q\cdot(q^{-1}\ \mathrm{mod}\ p)\right)M_p + \left(p\cdot(p^{-1}\ \mathrm{mod}\ q)\right)M_q\right)\ \mathrm{mod}\ n\right)^2\ \mathrm{mod}\ p\\
&=&\left(\left(\left(q\cdot(q^{-1}\ \mathrm{mod}\ p)\right)M_p + \left(p\cdot(p^{-1}\ \mathrm{mod}\ q)\right)M_q\right)\ \mathrm{mod}\ p\right)^2\ \mathrm{mod}\ p\\
&=&\left(M_p\ \mathrm{mod}\ p\right)^2\ \mathrm{mod}\ p\\
&=&M_p^2\ \mathrm{mod}\ p\\
&=&C\ \mathrm{mod}\ p
\end{eqnarray*}
e, analogamente 
$$
M_1^2\mod q = C\ \mathrm{mod}\ q
$$
Il <span style="color:gold">teorema cinese dei resti</span> assicura quindi che
$$
M_1^2\mod n = C\ \mathrm{mod}\ n
$$

* Per quanto riguarda $M_2$, banalmente
$$
M_2^2\mod n = (n-M_1)^2 \mod n = M_1^2\mod n = C\ \mathrm{mod}\ n
$$

* La dimostrazione per $M_3$ ed $M_4$ è identica

### <span style="color:cyan">Un semplice esempio</span>

In [ ]:
from ACLIB.utils import getprime, modexp, modular_inverse

In [ ]:
def rabinprime(N):
    '''Genera numeri primi n tali che n=4k+3'''
    while True:
        n = getprime(2**N)
        if n%4==3:
            return n

#### <span style="color:cyan">Generazione delle chiavi</span>

In [ ]:
nbytes=16
nbits=nbytes*8

In [ ]:
p = rabinprime(nbits>>1); p

In [ ]:
q = rabinprime(nbits>>1); q

In [ ]:
n=p*q; n

In [ ]:
cp = q*modular_inverse(q,p)
cq = p*modular_inverse(p,q)
print(cp,cq)

#### <span style="color:cyan">Cifratura</span>

In [ ]:
m=b'pippo'

In [ ]:
M = int.from_bytes(m,'big'); M

In [ ]:
C=(M*M)%n; C

#### <span style="color:cyan">Decifrazione</span>

In [ ]:
Mp = modexp(C,(p+1)>>2,p)
Mq = modexp(C,(q+1)>>2,q)
print(Mp,Mq)

In [ ]:
M1 = (cp*Mp+cq*Mq)%n
M2 = (cp*Mp-cq*Mq)%n
M3 = (-cp*Mp+cq*Mq)%n
M4 = (-cp*Mp-cq*Mq)%n

In [ ]:
# Per inciso...
Mi = (M1,M2,M3,M4)
[modexp(Mi[i],2,n) == C for i in range(4)]

In [ ]:
for i in range(4):
    print(Mi[i].to_bytes(16,'big'))

#### <span style="color:cyan">Equivalenza con il problema della fattorizzazione</span>
* Dobbiamo dimostrare che se siamo in grado di decifrare (con un'incertezza di 1 su 4) allora <span style="color:gold">possiamo anche fattorizzare $n$</span>
* Il <span style="color:gold">viceversa è ovvio</span>

* Abbiamo visto che le quattro radici di $C\mod n$ corrispondono alle <span style="color:gold">quattro possibili combinazioni</span> delle due radici modulo $p$ con le due radici modulo $q$
    1. $M_1$ corrisponde alla coppia <span style="color:gold">$(M_p,M_q)$</span>
    2. $M_2$ corrisponde alla coppia $(-M_p\mod p,-M_q\mod q)$&nbsp;=&nbsp;<span style="color:gold">$(p-M_p,q-M_q)$</span>
    3. $M_3$ corrisponde alla coppia $(M_p,-M_q\mod q)$&nbsp;=&nbsp;<span style="color:gold">$(M_p,q-M_q)$</span>
    4. $M_4$ corrisponde alla coppia $(-M_p\mod p,M_q)$&nbsp;=&nbsp;<span style="color:gold">$(p-M_p,M_q)$</span>
* Si consideri l'esempio di <span style="color:gold">$n=33=3\cdot 11$</span> in cui $4$ è ovviamente un residuo quadratico
* In tal caso abbiamo
$$
M_p = 4^{\frac{3+1}{4}}\mod 3=1\quad\mathrm{e}\quad M_q=4^{\frac{11+1}{4}}\mod 11=9
$$
* Risulta inoltre
$$
c_p=q\cdot(q^{-1}\ \mathrm{mod}\ p)=11\cdot(11^{-1}\mod 3)=11\cdot 2= 22
$$
e
$$
c_q=p\cdot(p^{-1}\ \mathrm{mod}\ q)=3\cdot(3^{-1}\mod 11)=3\cdot 4= 12
$$
* Possiamo ora evidenziare bene la corrispondenza:
    1. Ritroviamo $M_1=(c_p\cdot M_p+c_q\cdot M_q)\mod n = (22\cdot 1+12\cdot 9)\mod 33=130\mod 33=31$
    2. Poi $M_2=(c_p\cdot (p-M_p)+c_q\cdot (q-M_q))\mod n = 68\mod 33=2$
    3. $M_3=(c_p\cdot M_p+c_q\cdot (q-M_q))\mod n = 46\mod 33=13$
    4. $M_4=(c_p\cdot (p-M_p)+c_q\cdot M_q)\mod n = 152\mod 33=20$

In [ ]:
# verifica
modexp(31,2,33),modexp(2,2,33),modexp(13,2,33),modexp(20,2,33)

* Veniamo ora alla riduzione
* L'obiettivo è fattorizzare un numero $n$ dato in input e l'ipotesi è di disporre di un algoritmo <span style="color:gold">black-box</span> che, dato un residuo quadratico $C$ modulo $n$, restituisce una delle 4 possibili radici
di $C$ modulo $n$
* La riduzione è un <span style="color:gold">algoritmo tipo Las Vegas</span> e funziona nel modo seguente
    1. Genero a caso un numero <span style="color:gold">$r\in \zn$, $r\neq 0$</span> 
    2. Se $m=\mcd(r,n)\neq 1$ (la famosa <span style="color:gold">botta di fortuna</span>), restituisco $m$ e $n/m$
    3. Altrimenti considero $r$ come un "messaggio", calcolo $C=r^2\mod n$ e sottopongo <span style="color:gold">$C$ alla black box</span>
    4. Se $r'$ è il valore restituito dalla black-box, calcolo <span style="color:gold">$m=\mcd(r-r',n)$</span>
    5. Se $m>1$ e $m\neq n$ restituisco i <span style="color:gold">fattori $m$ e $n/m$</span>, altrimenti ritorno al passo 1

* La correttezza, ovvero il fatto che, con <span style="color:gold">probabilità strettamente positiva</span>, al passo 5 l'algoritmo trovi effettivamente un fattore di $n$, dipende proprio dalla corrispondenza delle radici con i residui modulo $p$ e modulo $q$
* Sappiamo che il valore $r'$ restituito dalla black box è <span style="color:gold">una delle quattro radici di $r^2$</span> e che ciascuna di esse corrisponde a uno dei quattro modi con con cui possiamo <span style="color:gold">combinare</span> le due radici di <span style="color:gold">$r^2$ modulo $p$</span> con le due radici di <span style="color:gold">$r^2$ modulo $q$</span>: si tratta dei 4 valori 
    * $r_p = r\mod p$, 
    * $-r_p = p-r_p$,
    * $r_q = r\mod q$,
    * $-r_q=q-r_q$.

* Supponiamo dunque, fissare le idee, che la radice <span style="color:gold">$r$ da cui siamo partiti corrisponda alla coppia $(r_p,r_q)$</span> e consideriamo le 4 possibili risposte dell'oracolo
    1. $r'=r$ (cioè anche $r'$ corrisponde alla coppia $(r_p,r_q)$). In tal caso $r-r'=0$ e $m=n$ e dunque, in questo caso, <span style="color:gold">l'algoritmo non dà risposta</span>;
    2. $r'$ corrisponde alla coppia $(p-r_p,q-r_q)$. Allora
    $$
    (r-r')\mod p = r_p-(p-r_p)\mod p=2r_p\mod p
    $$
    e analogamente 
    $$ 
    (r-r')\mod q=2r_q\mod q
    $$ da cui (sempre per il Teorema Cinese dei Resti) $r-r'\mod n=2r\mod n$. Allora necessariamente $\mcd(r-r',n)=1$ (altrimenti l'algoritmo si sarebbe fermato al passo 2) e pure in questo caso, l'algoritmo <span style="color:gold">non può dare risposta</span>
    3. $r'$ corrisponde alla coppia $(r_p,q-r_q)$. In questo caso abbiamo
    $$
    (r-r')\mod q = r_q-(q-r_q)\mod q=2r_q
    $$
    ma $(r-r')\mod p=0$ per cui $\mcd(r-r',n)=p$ e l'algoritmo <span style="color:gold">termina con successo</span>
    4. $r'$ corrisponde alla coppia $(p-r_p,r_q)$. Il caso è speculare al precedente e quindi l'algoritmo <span style="color:gold">termina con successo</span>
* Poiché $r$ è scelto a caso, l'oracolo (<span style="color:gold">fosse anche di tipo "malevolo"</span>) non può sapere a quale delle quattro radici corrisponde $C$ e dunque con <span style="color:gold">probabilità almeno 0.5</span> (in realtà un poco di più perché ci potrebbe essere il "colpo di fortuna" al passo 2) l'algoritmo termina un round con successo

In [ ]:
from random import choice
def oracle(p,q):
    '''Oracolo realizzato come coroutine'''
    n=p*q
    cp = q*modular_inverse(q,p)
    cq = p*modular_inverse(p,q)
    counter = 1
    C = yield "Oracle ready"
    while True:
        rp = modexp(C,(p+1)>>2,p)
        rq = modexp(C,(q+1)>>2,q)
        r1 = (cp*rp+cq*rq)%n
        r2 = n-r1
        r3 = (cp*rp-cq*rq)%n
        r4 = n-r3
        print(f"Invocazione {counter} dell'oracolo")
        counter += 1
        C = yield choice([r1,r2,r3,r4])

In [ ]:
from random import randint
from ACLIB.utils import Euclid

In [ ]:
N = 30
p = rabinprime(N)
q = rabinprime(N)
n = p*q
print(f"Il numero da fattorizzare è {n} e i fattori sono {p} e {q}")
O = oracle(p,q)     # creazione dell'oracolo
print(O.send(None)) # attivazione dell'oracolo, che "ora" è fermo a riga 8 in attesa di input
while True:
    r = randint(1,n-1)
    R = (r*r)%n
    print(f"Il numero casuale scelto è {r}")
    rp = O.send(R)  # l'oracolo riceve input, calcola, restituisce e si ferma a riga 18
    m = Euclid(r-rp,n)
    if m==n:
        print(f"L'oracolo ha restituito ancora rp = r = {r}")
    elif  m==1:
        print(f"L'oracolo ha restituito rp = n-r = {rp} ")
    else:
        print("L'oracolo ha restituito una radice utile per la fattorizzazione")
        break
    r = randint(1,n-1)
    R = (r*r)%n
# riveliamo ora p e q
print(f"I fattori calcolati sono {m} e {n//m}")

In [ ]:
from ACLIB.utils import modular_inverse

### <span style="color:gold">Il sistema crittografico a chiave pubblica di ElGamal</span>
* Si tratta di un sistema proposto da Taher ElGamal nel 1985, quindi <span style="color:gold">non è il primo crittosistema a chiave pubblica</span>.
* Esso segue di qualche anno il <span style="color:gold">sistema RSA</span>, brevettato nel 1977. 
* Lo trattiamo prima dell'RSA, e subito dopo il protocollo di DH per lo scambio i chiavi, in quanto è strettamente <span style="color:gold">legato a quest'ultimo</span>. 
* Poiché non è stato brevettato, per molto tempo è stato inserito, insieme al protocollo <span style="color:gold">DSA</span> (<span style="color:gold">Digital Signature Algorithm</span>), in diverse suite crittografiche, ad esempio <span style="color:gold">GNUPG</span>.

### <span style="color:cyan">Descrizione del protocollo</span>

* Come in qualsiasi protocollo crittografico a chiave pubblica, ci sono due momenti distinti:
    1. <span style="color:gold">generazione della coppia di chiavi</span>, pubblica e privata, con la conseguente "diffusione" (pubblicazione, appunto) della prima;
    2. esecuzione di comunicazioni cifrate fra le due parti, processo a sua volta composto da <span style="color:gold">cifratura e decifrazione</span>.
* Si noti che il protocollo è asimmetrico e dunque, per fissare le idee, considereremo la generazione delle chiavi dalla <span style="color:gold">sola parte di Alice</span>.


<br />

#### <span style="color:cyan">Generazione delle chiavi (Alice)</span>
1. Alice determina i parametri del protocollo: 
    * un <span style="color:gold">numero primo $p$</span> di lunghezza appropriata, 
    * una <span style="color:gold">radice primitiva $g$</span> di $\mathbf{Z}_p^*$, 
    * il valore $A=g^a\ \mathrm{mod}\ p$, dove $a\in \mathbf{Z}_p^*$ è un numero scelto <span style="color:gold">uniformemente a caso</span>. 
2. Alice conserva $a$ come propria <span style="color:gold">chiave segreta</span> e provvede alla diffusione della terna $(p,g,A)$ come corrispondente <span style="color:gold">chiave pubblica</span>.

<br />

* Per questa parte del protocollo, si può osservare che la differenza con DH è solo nei <span style="color:gold">destinatari della comunicazione</span> (non solo Bob bensì tutti coloro che dovranno inviare messaggi cifrati ad Alice).

<br />

#### <span style="color:cyan">Cifratura di un messaggio $M$ (Bob)</span>
1. Bob recupera la <span style="color:gold">chiave pubblica di Alice</span>: $(p,g,A)$;
2. sceglie un numero $b\in \mathbf{Z}_p^*$ <span style="color:gold">uniformemente a caso</span>;
3. calcola le <span style="color:gold">due quantità</span> $B=g^b\ \mathrm{mod}\ p$ e $c=\left(A^b\cdot M\right)\ \mathrm{mod}\ p$;
4. invia ad Alice la coppia $C=(B,c)$, che costituisce il <span style="color:gold">messaggio cifrato</span>.


<br />

#### <span style="color:cyan">Decifrazione del messaggio $C=(B,c)$ (Alice)</span>
1. Alice calcola la quantità $Z=B^a\mathrm{mod}\ p$;
2. calcola (usando l'algoritmo di Euclide esteso) $Z^{-1}\mod p$;
3. calcola $M = \left(Z^{-1}\cdot c\right)\mod p$.

* Si noti che il messaggio deve essere interpretabile come <span style="color:gold">numero minore di $p$</span>.

#### <span style="color:cyan">Correttezza</span>

* &Egrave; un'immediata conseguenza del fatto che $Z=B^a\mod p=A^b\mod p$ <span style="color:gold">esattamente come nel protocollo di Diffie-Hellman</span>

#### <span style="color:cyan">Efficienza</span>

* A parte la pre-computazione delle chiavi (operazione eseguita una sola volta), la complessità della cifratura è dominata da <span style="color:gold">due calcoli di potenze modulari</span> (cui va aggiunto un prodotto). 
* La decifrazione richiede invece il calcolo di <span style="color:gold">una potenza, di un inverso e di una moltiplicazione</span>. 
* Si tratta di poche operazioni che però sono eseguite su numeri di <span style="color:gold">oltre 1000 bit</span>. 
* Se il messaggio fosse più lungo della lunghezza del modulo $p$, Bob dovrebbe <span style="color:gold">spezzarlo in blocchi</span> e ripetere la cifratura, usando però un valore $k$ differente per ogni singolo blocco (vedremo subito perché). 
* Con molti blocchi, e cioè con messaggi lunghi, cifratura e decifrazione "asimmetriche" diventano in realtà <span style="color:gold">processi alquanto onerosi</span>, soprattutto se paragonati all'efficienza di moderni algoritmi di cifratura simmetrica (che possono anche avere supporto hardware).  
* Per questa ragione, la crittografia asimmetrica viene usata congiuntamente, e in modo sinergico, a quella simmetrica. 
* In particolare, protocolli asimmetrici vengono utilizzato in fase di <span style="color:gold">autenticazione delle parti</span> e per lo <span style="color:gold">scambio di chiavi</span> (come già abbiamo osservato). 
* Per la comunicazione vera e propria si utilizza invece un <span style="color:gold">protocollo simmetrico</span>.

#### <span style="color:cyan">Sicurezza</span>

* Il problema di violare lo schema di cifratura di ElGamal, cioè mettere in chiaro il messaggio $M$ a partire dai valori $p$, $g$, $A$, $b$ e $c$, è <span style="color:gold">equivalente a risolvere il CDH problem</span> (vedi cella succesiva).
* Se quindi vale l'ipotesi CDH, il <span style="color:gold">crittosistema di ElGamal è sicuro</span>.
* Un errore da non commettere è di utilizzare per cifrature differenti lo stesso valore $b$ (randomicamente scelto solo la prima volta).
* Se infatti si usa lo stesso valore di $b$, <span style="color:gold">anche $B$ non cambia</span> e dunque, a due messaggi distinti $M_1$ ed $M_2$, corrisponderebbero le due cifrature:

$$
C_1 = \left(B,c_1=\left(A^b\cdot M_1\right)\ \mathrm{mod}\ p\right)\qquad\mathrm{e}\qquad C_2 = \left(B,c_2=\left(A^b\cdot M_2\right)\ \mathrm{mod}\ p\right)
$$

* Moltiplicando, ad esempio, $c_1^{-1}\ \mathrm{mod}\ p$ per $c_2$, si vede che:
$$
c_1^{-1}\cdot c_2\ \mathrm{mod}\ p = (M_1^{-1}\cdot M_2)\ \mathrm{mod}\ p
$$
ovvero che
$$
M_2=\left(M_1\cdot \left(c_2\cdot c_1^{-1}\right)\right)\ \mathrm{mod}\ p
$$
Se dunque Eva fosse in grado (per qualsiasi motivo) di decifrare il messaggio $M_1$, potrebbe <span style="color:gold">decifrare anche i successivi messaggi cifrati con lo stesso valore $b$</span>.

#### <span style="color:cyan">$CDH\Rightarrow$ ElGamal è sicuro</span>
* Usiamo l'implicazione <span style="color:gold">contropositiva</span>: supponiamo cioè di poter "rompere" ElGamal e siamo in grado di risolvere l'assuzione CDH
* La dimostrazione è immediata nel momento in cui riflettiamo sul fatto che ElGamal "oscura" il messaggio moltiplicandolo (modulo $p$) proprio per una quantità $A^b\mod p$, che corrisponde al <span style="color:gold">segreto condiviso di Diffie-Hellman</span>.
* Se dunque siamo in grado di mettere in chiaro $M$ (senza il calcolo diretto del logaritmo discreto $a = \log_g A$), possiamo risalire alla quantità <span style="color:gold">$A^b\mod p=g^{a\cdot b}\mod p$</span>, proprio <span style="color:gold">la quantità che la CDH assumption chiede di calcolare</span>.

#### <span style="color:cyan">Qualche esperimento con la libreria Crypto di Python </span>
* Cifratura e firma digitale <span style="color:gold">non sono più supportate in PyCryptodome</span>
* Rimane disponibile la <span style="color:gold">creazione di chiavi</span>
* Come esercizio, forniamo qui una <span style="color:gold">"textbook version"</span> di encrypt e decrypt

In [ ]:
from Crypto.PublicKey import ElGamal
from Crypto.Random.random import Random
from Crypto.Hash import SHA
from Crypto.Math import Numbers

In [ ]:
key = ElGamal.generate(256, Random.get_random_bytes)

In [ ]:
key.__dict__

* x e y sono le quantità che, nella descrizione del protocollo, abbiamo indicato con $a$ e $A$

In [ ]:
class EGKey(ElGamal.ElGamalKey):
    '''Define ElGamal key with "textbook" encryption/decryption implemented'''
    def __init__(self,l,randfun=Random.get_random_bytes):
        '''The super().__init__ method would be useless here. pycryptodome includes a
           function to generate ElGamal keys. We generate one and copy
           the dictionary to self.
           Simply returning the generated key would give an ElGamal key
           and not an EGKey'''
        self.__dict__ = ElGamal.generate(l,randfun).__dict__
    def publickey(self):
        '''Returns an EGKey public key corresponding to self
           (i.e. without the secret information)
        '''
        pubkey = super().__new__(EGKey)   # create an "empty" ElGamalKey
        pubkey.p = self.p
        pubkey.g = self.g
        pubkey.y = self.y
        return pubkey
    def decrypt(self,ciphertext):
        '''Decrypt cipertext using self key'''
        B = Numbers.Integer(ciphertext[0])
        c = Numbers.Integer(ciphertext[1])
        B.inplace_pow(self.x,self.p)
        M = (B.inverse(self.p)*c)%self.p
        return M.to_bytes(self.p.size_in_bytes())
    def encrypt(self,plaintext):
        '''Encrypt plaintext using self key'''
        assert len(plaintext) <= self.p.size_in_bytes()
        while (b:=Numbers.Integer.random_range(min_inclusive=1,max_inclusive=self.p)) \
               and b.gcd(self.p-1)!=1:
            pass
        g = Numbers.Integer(self.g) 
        A = Numbers.Integer(self.y)
        M = Numbers.Integer.from_bytes(plaintext)
        A.inplace_pow(b,self.p)
        return g.inplace_pow(b,self.p),(A*M)%self.p

In [ ]:
mykey = EGKey(256)

In [ ]:
mykey.__dict__

In [ ]:
pubkey = mykey.publickey()

In [ ]:
pubkey.__dict__

In [ ]:
message = b"Quite a short text"

In [ ]:
ciphertext = pubkey.encrypt(message);ciphertext

In [ ]:
mykey.decrypt(ciphertext)

#### <span style="font-style: italic; color:cyan">Esercizio</span>
* Modificare le funzioni di cifratura e decifrazione in modo da evitare i <span style="font-style: italic; color:gold">byte di padding</span> nel testo rimesso in chiaro